đúng

In [7]:
import cv2
import numpy as np
from datetime import datetime

grid_size = (5, 5)
dai, rong = grid_size


def create_edges(positions):
    '''  Tạo cạnh từ danh sách đỉnh '''
    result = []
    for i in range(0, len(positions) - 1):
        item = positions[i]
        next_item = positions[i+1]
        if item[0] == next_item[0]:
            if item[1] < next_item[1]:
                # dọc thuận - done
                for i in range(item[1], next_item[1] + 1):
                    result.append([item[0], i])
            if item[1] > next_item[1]:
                # dọc ngược - done
                for i in range(next_item[1], item[1] + 1):
                    result.append([item[0], i])                
        
        elif item[1] == next_item[1]:    
            if item[0] < next_item[0]:
                # ngang thuan - done
                for i in range(item[0], next_item[0] + 1):
                    result.append([i, item[1]])         
            if item[0] > next_item[0]:
                # ngang nguoc - done
                for i in range(next_item[0], item[0] + 1):
                        result.append([i, item[1]])
    return result


# positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
#                       [430, 100], [520, 100], [520, 150],[600, 150], [600, 200], [520, 200]
#                       , [520, 440], [650, 440], [650, 630],
#                       [210, 630], [210, 530], [70, 530], [70, 420],
#                         [150, 420], [150, 140], [50, 140], [50, 40]]

positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
                      [430, 100], [520, 100], [520, 150],[600, 150], [600, 220], [620, 220],
                      [620, 180], [660, 180], [660, 280], 
                      [550,280], [550,220], [520, 220], 
                      [520, 440], [650, 440], [650, 630],
                      [210, 630], [210, 530], [70, 530], [70, 420], 
                      [150, 420], [150, 140], [50, 140], [50, 40]]
# positions = np.array([[60, 60], [270, 60], [270, 270], [60, 270], [60, 60]])
# positions = [[50, 40], [150, 40], [150, 120], [300, 120], [300, 240], [50, 240], [50, 40]]

In [20]:
image = np.ones((720, 720, 3), np.uint8) * 255

def get_point_in_shape(grid_size, positions):
    ''' Vẽ lưới mới từ tâm của cells edge'''
    cells_edge = create_edges(positions)
    current_datetime = datetime.now()
    print(f"BẮT ĐẦU: {current_datetime}")
    trung_diem = [int(np.mean(np.array(cells_edge)[:,0])), int(np.mean(np.array(cells_edge)[:,1]))]
    
    l1, l2, l3, l4 = [], [], [], []
    check1, check2, check3, check4 = [], [], [], []
    check11, check22, check33, check44 = [], [], [], []
    
    # TẠO CÁC DANH SÁCH ĐỈNH CẠNH VÀ ĐƯỜNG BAO
    
    for [x, y] in cells_edge:
        # 1
        if x >= trung_diem[0] and y <= trung_diem[1]:
            check1.append([x, y])
            check11.append([x, y])
            if [x, y] in positions:
                check1.append([x, y])
        
        # 2
        if x <= trung_diem[0] and y <= trung_diem[1]:
           check2.append([x, y])
           check22.append([x, y])
           if [x, y] in positions:
               check2.append([x, y])
        
        # 3
        if x > trung_diem[0] and y > trung_diem[1]:
            check3.append([x, y])
            check33.append([x, y])
            if [x,y] in positions:
                check3.append([x, y])
            
        # 4     
        if x < trung_diem[0] and y > trung_diem[1]:
            check4.append([x,y])
            check44.append([x,y])
            if [x, y] in positions:
                check4.append([x,y])
    
    minmax_x = [min(np.array(cells_edge)[:,0]),max(np.array(cells_edge)[:,0])]
    minmax_y = [min(np.array(cells_edge)[:,1]),max(np.array(cells_edge)[:,1])]
    
    # # 1
    for x in range(trung_diem[0], minmax_x[1]+1, 1):
        for y in range(trung_diem[1], minmax_y[0]-1, -1):
            if ((x - trung_diem[0]) % grid_size[0] == 0 or (trung_diem[1] - y) % grid_size[1] == 0):  # là vị trí đường lưới
                arrX_item = [i for [i, j] in cells_edge if j == y]
                arrY_item = [j for [i, j] in cells_edge if i == x]
                if arrY_item and arrX_item:
                    mmx = [min(arrX_item), max(arrX_item)]
                    mmy = [min(arrY_item), max(arrY_item)]
                    if ((x >= mmx[0]) and (x <= mmx[1])) and (y >= mmy[0] and y <= mmy[1]):  # bên trong
                        if x == trung_diem[0] or y == trung_diem[1]:
                            check11.append([x,y])
                        if ((x - trung_diem[0]) % grid_size[0] == 0 or (trung_diem[1] - y) % grid_size[1] == 0): # nằm trên lưới
                            check1.append([x, y])
                            if ((x - trung_diem[0]) % grid_size[0] == 0 and (trung_diem[1] - y) % grid_size[1] == 0): # điểm cắt giữa đường dọc và ngang
                                check1.append([x, y])
    
    # # 2
    for x in range(trung_diem[0], minmax_x[0] - 1, -1):
        for y in range(trung_diem[1], minmax_y[0] - 1, -1):
            if ((trung_diem[0] - x) % grid_size[0] == 0 or (trung_diem[1] - y) % grid_size[1] == 0):
                arrX_item = [i for [i, j] in cells_edge if j == y]
                arrY_item = [j for [i, j] in cells_edge if i == x]
                if arrY_item and arrX_item:  # bên trong
                    mmx = [min(arrX_item), max(arrX_item)]
                    mmy = [min(arrY_item), max(arrY_item)]
                    if ((x >= mmx[0]) and (x <= mmx[1])) and (y >= mmy[0] and y <= mmy[1]):
                        if x == trung_diem[0] or y == trung_diem[1]:
                            check22.append([x,y])
                        if ((trung_diem[0] - x) % grid_size[0] == 0 or (trung_diem[1] - y) % grid_size[1] == 0):
                            check2.append([x, y])
                            if ((trung_diem[0] - x) % grid_size[0] == 0 and (trung_diem[1] - y) % grid_size[1] == 0):
                                check2.append([x, y])
                                
    # 3
    for x in range(trung_diem[0], minmax_x[1]+ 1, 1):
        for y in range(trung_diem[1], minmax_y[1] + 1, 1):
            if ((x - trung_diem[0]) % grid_size[0] == 0 or (y - trung_diem[1]) % grid_size[1] == 0):
                arrX_item = [i for [i, j] in cells_edge if j == y]
                arrY_item = [j for [i, j] in cells_edge if i == x]
                if arrY_item and arrX_item:  # bên trong
                    mmx = [min(arrX_item), max(arrX_item)]
                    mmy = [min(arrY_item), max(arrY_item)]
                    if ((x >= mmx[0]) and (x <= mmx[1])) and (y >= mmy[0] and y <= mmy[1]):
                        if x == trung_diem[0] or y == trung_diem[1]:
                            check33.append([x,y])
                        if ((x - trung_diem[0]) % grid_size[0] == 0 or (y - trung_diem[1]) % grid_size[1] == 0):
                            check3.append([x, y])
                            if ((x - trung_diem[0]) % grid_size[0] == 0 and (y - trung_diem[1]) % grid_size[1] == 0):
                                check3.append([x, y])
    
    # # 4
    for x in range(trung_diem[0], minmax_x[0] - 1, - 1):
        for y in range(trung_diem[1], minmax_y[1] + 1, 1):
            if (trung_diem[0] - x) % grid_size[0] == 0 or (y - trung_diem[1]) % grid_size[1] == 0:  
                arrX_item = [i for [i, j] in cells_edge if j == y]
                arrY_item = [j for [i, j] in cells_edge if i == x]
                if arrY_item and arrX_item:  # bên trong
                    mmx = [min(arrX_item), max(arrX_item)]
                    mmy = [min(arrY_item), max(arrY_item)]
                    if ((x >= mmx[0]) and (x <= mmx[1])) and (y >= mmy[0] and y <= mmy[1]):  
                        if x == trung_diem[0] or y == trung_diem[1]:
                            check44.append([x,y])
                        if (trung_diem[0] - x) % grid_size[0] == 0 or (y - trung_diem[1]) % grid_size[1] == 0:
                            check4.append([x, y])
                            if (trung_diem[0] - x) % grid_size[0] == 0 and (y - trung_diem[1]) % grid_size[1] == 0:
                                check4.append([x, y])
                      
    dinh1 = [[x, y] for [x, y] in check1 if check1.count([x, y]) == 2]
    dinh2 = [[x, y] for [x, y] in check2 if check2.count([x, y]) == 2]
    dinh3 = [[x, y] for [x, y] in check3 if check3.count([x, y]) == 2]
    dinh4 = [[x, y] for [x, y] in check4 if check4.count([x, y]) == 2]
    
    # loại bỏ các index giống nhau
    dinh1 = set(map(tuple, dinh1))
    dinh1 = list(map(list, dinh1))
    dinh2 = set(map(tuple, dinh2))
    dinh2 = list(map(list, dinh2))
    dinh3 = set(map(tuple, dinh3))
    dinh3 = list(map(list, dinh3))
    dinh4 = set(map(tuple, dinh4))
    dinh4 = list(map(list, dinh4))
    
    # hợp danh sách đường đường lưới và đường bao
    canh1 = [[x, y] for [x, y] in check1 if check1.count([x, y]) == 1] + dinh1
    canh2 = [[x, y] for [x, y] in check2 if check2.count([x, y]) == 1] + dinh2
    canh3 = [[x, y] for [x, y] in check3 if check3.count([x, y]) == 1] + dinh3
    canh4 = [[x, y] for [x, y] in check4 if check4.count([x, y]) == 1] + dinh4
    
    
    c0 = datetime.now()
    print(f"Xong 0: {(c0 - current_datetime).total_seconds()}s")

    # TẠO MA TRẬN
    #####################################################################################################################
    # DONE 1
    # Phải - Trên
    for j in range(trung_diem[1], minmax_y[0]-1, -1):
        arr = []
        for i in range(trung_diem[0], minmax_x[1]+1, 1):
            cell = [i, j, None, None]
            arrX_item = [x for [x, y] in check11 if y == j]
            arrY_item = [y for [x, y] in check11 if x == i]
            if arrY_item and arrX_item:     
                mmx = [min(arrX_item), max(arrX_item)]
                mmy = [min(arrY_item), max(arrY_item)]
                if ((i >= mmx[0]) and (i <= mmx[1])) and (j >= mmy[0] and j <= mmy[1]):  # thịt
                    if [i, j] not in canh1:
                        cell = [i, j, 0, None]
                    else:
                        cell = [i, j, 1, None]
                    if [i, j] in dinh1:
                        cell = [i, j, 2, None]
            arr.append(cell)
        l1.append(arr)
                    
    c1 = datetime.now()
    print(f"Xong 1: {(c1 - c0).total_seconds()}s")
                   
    # DONE
    # Trái - Trên
    # # ngang
    for i in range(trung_diem[0], minmax_x[0] - 1, -1):
        arr = []
        for j in range(trung_diem[1], minmax_y[0] - 1, -1):
            cell = [i, j, None, None]
            arrX_item = [x for [x, y] in check22 if y == j]
            arrY_item = [y for [x, y] in check22 if x == i]
            if arrY_item and arrX_item:
                mmx = [min(arrX_item), max(arrX_item)]
                mmy = [min(arrY_item), max(arrY_item)]
                if ((i >= mmx[0]) and (i <= mmx[1])) and (j >= mmy[0] and j <= mmy[1]):
                    if [i, j] not in canh2:
                        cell = [i, j, 0, None]
                    else:
                        cell = [i, j, 1, None]
                    if [i, j] in dinh2:
                        cell = [i, j, 2, None]
            arr.append(cell)
        l2.append(arr)
            
    c2 = datetime.now()
    print(f"Xong 2: {(c2 - c1).total_seconds()}s")
    
    # # DONE
    # # Phải - Dưới
    # # dọc
    for i in range(trung_diem[0], minmax_x[1]+ 1, 1):
        arr = []
        for j in range(trung_diem[1], minmax_y[1] + 1, 1):
            cell = [i, j, None, None]
            arrX_item = [x for [x, y] in check33 if y == j]
            arrY_item = [y for [x, y] in check33 if x == i]
            if arrY_item and arrX_item:
                mmx = [min(arrX_item), max(arrX_item)]
                mmy = [min(arrY_item), max(arrY_item)]
                if ((i >= mmx[0]) and (i <= mmx[1])) and (j >= mmy[0] and j <= mmy[1]):
                    if [i, j] not in canh3:
                        cell = [i, j, 0, None]
                    else:
                        cell = [i, j, 1, None]
                    if [i, j] in dinh3:
                        cell = [i, j, 2, None]
            arr.append(cell)
        l3.append(arr)
    
    c3 = datetime.now()
    print(f"Xong 3: {(c3 - c2).total_seconds()}s")
                            
    # # DONE             
    # # Trái - Dưới
    for i in range(trung_diem[0], minmax_x[0] - 1, - 1):
        arr = []
        for j in range(trung_diem[1], minmax_y[1] + 1, 1):
            cell = [i, j, None, None]
            arrX_item = [x for [x, y] in check44 if y == j]
            arrY_item = [y for [x, y] in check44 if x == i]
            if arrY_item and arrX_item:     
                mmx = [min(arrX_item), max(arrX_item)]
                mmy = [min(arrY_item), max(arrY_item)]
                if ((i >= mmx[0]) and (i <= mmx[1])) and (j >= mmy[0] and j <= mmy[1]):  # bên trong
                    if [i, j] not in canh4:
                        cell = [i, j, 0, None]
                    else:
                        cell = [i, j, 1, None]
                    if [i, j] in dinh4:
                        cell = [i, j, 2, None]
            arr.append(cell)
        l4.append(arr)
    
    c4 = datetime.now()
    print(f"Xong 4: {(c4 - c3).total_seconds()}s")
    print(f"TỔNG CỘNG: {(c4 - c0).total_seconds()}s")
    
    return l1, l2, l3, l4, check11, check22, check33, check44, dinh1, dinh2, dinh3, dinh4, check1, check2, check3, check4, check11, check22, check33, check44

new_gs = [60, 90]
cells1, cells2, cells3, cells4, canh_1, canh_2, canh_3, canh_4, dinh1, dinh2, dinh3, dinh4, check1, check2, check3, check4, check11, check22, check33, check44 = get_point_in_shape(new_gs, positions)

BẮT ĐẦU: 2024-07-08 11:49:49.120867
Xong 0: 5.043298s
Xong 1: 15.656636s
Xong 2: 36.576549s
Xong 3: 27.651351s
Xong 4: 38.482299s
TỔNG CỘNG: 118.366835s


In [21]:
image = np.ones((720, 720, 3), np.uint8) * 255
result1 = cells1
result2 = cells2
result3 = cells3
result4 = cells4
index = 0
cells_shape1 = np.array(result1).shape
cells_shape2 = np.array(result2).shape
cells_shape3 = np.array(result3).shape
cells_shape4 = np.array(result4).shape

# 1
for i in range(0, cells_shape1[1]-1):
    for j in range(0, cells_shape1[0]-1):
        cell = result1[j][i]
        cell_trai = result1[j][i - 1]
        cell_duoi = result1[j-1][i]
        cell_duoitrai = result1[j-1][i-1]
        if cell[2] == 0:
            if cell_duoitrai[3] == None and cell_duoi[2] == 1 and cell_trai[2] == 1:
                # cv2.line(image, (cell[0], cell[1]), (cell[0], cell[1]), (0,0,0), 10)
                # cv2.line(image, (cell_duoitrai[0], cell_duoitrai[1]), (cell_duoitrai[0], cell_duoitrai[1]), (0,0,255), 1)
                result1[j][i][3] = index
                index += 1
            if cell_trai[3] != None:
                result1[j][i][3] = cell_trai[3]
                # cv2.line(image, (cell[0], cell[1]), (cell[0], cell[1]), (0,0,255), 1)
            if cell_duoi[3] != None:
                result1[j][i][3] = cell_duoi[3]
                
# 2
for i in range(0, cells_shape2[1]-1):
    for j in range(0, cells_shape2[0]-1):
        cell = result2[j][i]
        cell_trai = result2[j][i - 1]
        cell_duoi = result2[j-1][i]
        cell_duoitrai = result2[j-1][i-1]
        if cell[2] == 0:
            if cell_duoitrai[3] == None and cell_duoi[2] == 1 and cell_trai[2] == 1:
                # cv2.line(image, (cell[0], cell[1]), (cell[0], cell[1]), (0,0,0), 10)
                # cv2.line(image, (cell_duoitrai[0], cell_duoitrai[1]), (cell_duoitrai[0], cell_duoitrai[1]), (0,0,255), 1)
                result2[j][i][3] = index
                index += 1
            if cell_trai[3] != None:
                result2[j][i][3] = cell_trai[3]
                # cv2.line(image, (cell[0], cell[1]), (cell[0], cell[1]), (0,0,255), 1)
            if cell_duoi[3] != None:
                result2[j][i][3] = cell_duoi[3]

# 3
for i in range(0, cells_shape3[1]-1):
    for j in range(0, cells_shape3[0]-1):
        cell = result3[j][i]
        cell_trai = result3[j][i - 1]
        cell_duoi = result3[j-1][i]
        cell_duoitrai = result3[j-1][i-1]
        if cell[2] == 0:
            if cell_duoitrai[3] == None and cell_duoi[2] == 1 and cell_trai[2] == 1:
                # cv2.line(image, (cell[0], cell[1]), (cell[0], cell[1]), (0,0,0), 10)
                # cv2.line(image, (cell_duoitrai[0], cell_duoitrai[1]), (cell_duoitrai[0], cell_duoitrai[1]), (0,0,255), 1)
                result3[j][i][3] = index
                index += 1
            if cell_trai[3] != None:
                result3[j][i][3] = cell_trai[3]
                # cv2.line(image, (cell[0], cell[1]), (cell[0], cell[1]), (0,0,255), 1)
            if cell_duoi[3] != None:
                result3[j][i][3] = cell_duoi[3]
                
# 4
for i in range(0, cells_shape4[1]-1):
    for j in range(0, cells_shape4[0]-1):
        cell = result4[j][i]
        cell_trai = result4[j][i - 1]
        cell_duoi = result4[j-1][i]
        cell_duoitrai = result4[j-1][i-1]
        if cell[2] == 0:
            if cell_duoitrai[3] == None and cell_duoi[2] == 1 and cell_trai[2] == 1:
                # cv2.line(image, (cell[0], cell[1]), (cell[0], cell[1]), (0,0,0), 10)
                # cv2.line(image, (cell_duoitrai[0], cell_duoitrai[1]), (cell_duoitrai[0], cell_duoitrai[1]), (0,0,255), 1)
                result4[j][i][3] = index
                index += 1
            if cell_trai[3] != None:
                result4[j][i][3] = cell_trai[3]
                # cv2.line(image, (cell[0], cell[1]), (cell[0], cell[1]), (0,0,255), 1)
            if cell_duoi[3] != None:
                result4[j][i][3] = cell_duoi[3]

# gom nhóm các đỉnh của mảnh nhỏ
result_11 = []
result_22 = []
result_33 = []
result_44 = []

### 1
for i in range(1, cells_shape1[1]-1):
    for j in range(1, cells_shape1[0]-1):
        cell = result1[j][i]
        cell_trai = result1[j][i - 1]
        cell_phai = result1[j][i + 1]
        cell_duoi = result1[j-1][i]
        cell_tren = result1[j+1][i]
        cell_duoitrai = result1[j-1][i-1]
        cell_trentrai = result1[j+1][i-1]
        cell_trenphai = result1[j+1][i+1]
        cell_duoiphai = result1[j-1][i+1]
        
        if cell[2] == 0:
            if cell_duoitrai[2] == 2:
                result_11.append([cell_duoitrai[0], cell_duoitrai[1], cell_duoitrai[2] , cell[3]])
                # cv2.line(image, (cell_duoitrai[0], cell_duoitrai[1]), (cell_duoitrai[0], cell_duoitrai[1]), (0,0,255), 3)
            if cell_trentrai[2] == 2:
                result_11.append([cell_trentrai[0], cell_trentrai[1], cell_trentrai[2] , cell[3]])
                # cv2.line(image, (cell_trentrai[0], cell_trentrai[1]), (cell_trentrai[0], cell_trentrai[1]), (255,0,255), 3)
            if cell_trenphai[2] == 2:
                result_11.append([cell_trenphai[0], cell_trenphai[1], cell_trenphai[2] , cell[3]])
                # cv2.line(image, (cell_trenphai[0], cell_trenphai[1]), (cell_trenphai[0], cell_trenphai[1]), (0,0,0), 3)
            if cell_duoiphai[2] == 2:
                result_11.append([cell_duoiphai[0], cell_duoiphai[1], cell_duoiphai[2] , cell[3]])
                # cv2.line(image, (cell_duoiphai[0], cell_duoiphai[1]), (cell_duoiphai[0], cell_duoiphai[1]), (255,0,0), 3)

### 2
for i in range(1, cells_shape2[1]-1):
    for j in range(1, cells_shape2[0]-1):
        cell = result2[j][i]
        cell_trai = result2[j][i - 1]
        cell_phai = result2[j][i + 1]
        cell_duoi = result2[j-1][i]
        cell_tren = result2[j+1][i]
        cell_duoitrai = result2[j-1][i-1]
        cell_trentrai = result2[j+1][i-1]
        cell_trenphai = result2[j+1][i+1]
        cell_duoiphai = result2[j-1][i+1]
        
        if cell[2] == 0:
            if cell_duoitrai[2] == 2:
                result_22.append([cell_duoitrai[0], cell_duoitrai[1], cell_duoitrai[2] , cell[3]])
                # cv2.line(image, (cell_duoitrai[0], cell_duoitrai[1]), (cell_duoitrai[0], cell_duoitrai[1]), (0,0,255), 3)
            if cell_trentrai[2] == 2:
                result_22.append([cell_trentrai[0], cell_trentrai[1], cell_trentrai[2] , cell[3]])
                # cv2.line(image, (cell_trentrai[0], cell_trentrai[1]), (cell_trentrai[0], cell_trentrai[1]), (255,100,255), 3)
            if cell_trenphai[2] == 2:
                result_22.append([cell_trenphai[0], cell_trenphai[1], cell_trenphai[2] , cell[3]])
                # cv2.line(image, (cell_trenphai[0], cell_trenphai[1]), (cell_trenphai[0], cell_trenphai[1]), (0,0,0), 3)
            if cell_duoiphai[2] == 2:
                result_22.append([cell_duoiphai[0], cell_duoiphai[1], cell_duoiphai[2] , cell[3]])
                # cv2.line(image, (cell_duoiphai[0], cell_duoiphai[1]), (cell_duoiphai[0], cell_duoiphai[1]), (255,0,0), 3)
            
### 3
for i in range(1, cells_shape3[1]-1):
    for j in range(1, cells_shape3[0]-1):
        cell = result3[j][i]
        cell_trai = result3[j][i - 1]
        cell_phai = result3[j][i + 1]
        cell_duoi = result3[j-1][i]
        cell_tren = result3[j+1][i]
        cell_duoitrai = result3[j-1][i-1]
        cell_trentrai = result3[j+1][i-1]
        cell_trenphai = result3[j+1][i+1]
        cell_duoiphai = result3[j-1][i+1]
        
        if cell[2] == 0:
            if cell_duoitrai[2] == 2:
                result_33.append([cell_duoitrai[0], cell_duoitrai[1], cell_duoitrai[2] , cell[3]])
                # cv2.line(image, (cell_duoitrai[0], cell_duoitrai[1]), (cell_duoitrai[0], cell_duoitrai[1]), (0,0,255), 3)
            if cell_trentrai[2] == 2:
                result_33.append([cell_trentrai[0], cell_trentrai[1], cell_trentrai[2] , cell[3]])
                # cv2.line(image, (cell_trentrai[0], cell_trentrai[1]), (cell_trentrai[0], cell_trentrai[1]), (255,100,255), 3)
            if cell_trenphai[2] == 2:
                result_33.append([cell_trenphai[0], cell_trenphai[1], cell_trenphai[2] , cell[3]])
                # cv2.line(image, (cell_trenphai[0], cell_trenphai[1]), (cell_trenphai[0], cell_trenphai[1]), (0,0,0), 3)
            if cell_duoiphai[2] == 2:
                result_33.append([cell_duoiphai[0], cell_duoiphai[1], cell_duoiphai[2] , cell[3]])
                # cv2.line(image, (cell_duoiphai[0], cell_duoiphai[1]), (cell_duoiphai[0], cell_duoiphai[1]), (255,0,0), 3)
            
### 3
for i in range(1, cells_shape4[1]-1):
    for j in range(1, cells_shape4[0]-1):
        cell = result4[j][i]
        cell_trai = result4[j][i - 1]
        cell_phai = result4[j][i + 1]
        cell_duoi = result4[j-1][i]
        cell_tren = result4[j+1][i]
        cell_duoitrai = result4[j-1][i-1]
        cell_trentrai = result4[j+1][i-1]
        cell_trenphai = result4[j+1][i+1]
        cell_duoiphai = result4[j-1][i+1]
        
        if cell[2] == 0:
            if cell_duoitrai[2] == 2:
                result_44.append([cell_duoitrai[0], cell_duoitrai[1], cell_duoitrai[2] , cell[3]])
                # cv2.line(image, (cell_duoitrai[0], cell_duoitrai[1]), (cell_duoitrai[0], cell_duoitrai[1]), (0,0,255), 3)
            if cell_trentrai[2] == 2:
                result_44.append([cell_trentrai[0], cell_trentrai[1], cell_trentrai[2] , cell[3]])
                # cv2.line(image, (cell_trentrai[0], cell_trentrai[1]), (cell_trentrai[0], cell_trentrai[1]), (255,100,255), 3)
            if cell_trenphai[2] == 2:
                result_44.append([cell_trenphai[0], cell_trenphai[1], cell_trenphai[2] , cell[3]])
                # cv2.line(image, (cell_trenphai[0], cell_trenphai[1]), (cell_trenphai[0], cell_trenphai[1]), (0,0,0), 3)
            if cell_duoiphai[2] == 2:
                result_44.append([cell_duoiphai[0], cell_duoiphai[1], cell_duoiphai[2] , cell[3]])
                # cv2.line(image, (cell_duoiphai[0], cell_duoiphai[1]), (cell_duoiphai[0], cell_duoiphai[1]), (255,0,0), 3)

from itertools import groupby

# Thịt
array_1 = [item for item in np.array(result1).reshape(-1,4).tolist() if item[3] != None]
r_1 = sorted(array_1, key= lambda x: x[3])
array_2 = [item for item in np.array(result2).reshape(-1,4).tolist() if item[3] != None]
r_2 = sorted(array_2, key= lambda x: x[3])
array_3 = [item for item in np.array(result3).reshape(-1,4).tolist() if item[3] != None]
r_3 = sorted(array_3, key= lambda x: x[3])
array_4 = [item for item in np.array(result4).reshape(-1,4).tolist() if item[3] != None]
r_4 = sorted(array_4, key= lambda x: x[3])

r_ = r_1 + r_2 + r_3 + r_4
groups_thit = [list(group) for key, group in groupby(r_, key=lambda x: x[3])]
groups_thit = sorted(groups_thit, key=len, reverse=True)

# góc cạnh
canh_1 = [item for item in np.array(result1).reshape(-1,4).tolist() if item[2] == 1 or item[2] == 2]
canh_2 = [item for item in np.array(result2).reshape(-1,4).tolist() if item[2] == 1 or item[2] == 2]
canh_3 = [item for item in np.array(result3).reshape(-1,4).tolist() if item[2] == 1 or item[2] == 2]
canh_4 = [item for item in np.array(result4).reshape(-1,4).tolist() if item[2] == 1 or item[2] == 2]

groups_canh = canh_1 + canh_2 + canh_3 + canh_4
result_all = result_11 + result_22 + result_33 + result_44
result_all = sorted(result_all, key=lambda x: x[3])
result = [list(group) for key, group in groupby(result_all, key= lambda x: x[3])]

import random as rd
import pandas as pd


# Vẽ biểu đồ đường thẳng
color = (255,0,0)
check = len(groups_thit[0])

for arr in groups_thit:
    if check - len(arr) != 0:
        color = (0, rd.randint(0,250), rd.randint(200,255))
        check = len(arr)
        
    for [x,y,val,gr] in arr:
        if val == 0:
            cv2.line(image, (x,y), (x,y), color,1)
    
for [x,y,val,rg] in groups_canh:
    if val == 1:
        cv2.line(image, (x,y), (x,y), (0,0,0),1)

print('danh sách tọa độ\n', len(result))
for item in result:
    print(len(item),item)
    
for arr in result:
    for x,y,val,gr in arr:
        if val == 2:
            cv2.line(image, (x,y), (x,y), (0,0,255),3)       
cells_edge = create_edges(positions)     
trung_diem = [int(np.mean(np.array(cells_edge)[:,0])), int(np.mean(np.array(cells_edge)[:,1]))]
x,y = trung_diem
cv2.line(image, (x,y), (x,y), (0,0,0),10)       

for [x, y] in positions:
    cv2.line(image, (x,y), (x,y), (255,50,100),5)
    
for [x, y] in cells_edge:
    cv2.line(image, (x,y), (x,y), (0,0,0),1)

while True:
    cv2.imshow('DatGach', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()

[[388, 300, 2, 0], [388, 210, 2, 0], [388, 210, 2, 1], [388, 120, 2, 1], [388, 120, 2, 2], [388, 40, 2, 2], [448, 300, 2, 0], [448, 210, 2, 0], [448, 210, 2, 1], [448, 120, 2, 1], [448, 120, 2, 2], [448, 100, 2, 2], [448, 60, 2, 2], [448, 40, 2, 2], [448, 300, 2, 3], [448, 210, 2, 3], [448, 210, 2, 4], [448, 120, 2, 4], [448, 120, 2, 5], [448, 100, 2, 5], [448, 60, 2, 6], [448, 40, 2, 6], [508, 300, 2, 3], [508, 210, 2, 3], [508, 210, 2, 4], [508, 120, 2, 4], [508, 120, 2, 5], [508, 100, 2, 5], [508, 60, 2, 6], [508, 40, 2, 6], [508, 300, 2, 7], [508, 210, 2, 7], [508, 210, 2, 8], [508, 120, 2, 8], [508, 120, 2, 9], [508, 100, 2, 9], [508, 60, 2, 10], [508, 40, 2, 10], [520, 300, 2, 7], [520, 120, 2, 8], [520, 120, 2, 9], [568, 280, 2, 12], [568, 210, 2, 12], [568, 210, 2, 8], [568, 150, 2, 8], [568, 60, 2, 10], [568, 40, 2, 10], [568, 280, 2, 13], [568, 210, 2, 13], [568, 210, 2, 14], [568, 150, 2, 14], [568, 60, 2, 15], [568, 40, 2, 15], [600, 210, 2, 13], [600, 210, 2, 14], [620, 21

In [11]:
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# import numpy as np
# import random as rd

# fig, ax = plt.subplots(figsize=(12,12))

# # Vẽ biểu đồ đường thẳng
# ax.plot(np.array(positions)[:,0], np.array(positions)[:,1], color='red', marker=',', label='Khuôn', lw=2)
# color = [k for (k,v) in mcolors.CSS4_COLORS.items()]

# ind = len(groups_thit[0])
# color_ind = 2
# for arr in groups_thit:
#     if len(arr) - ind != 0:
#         ind = len(arr)
#         color_ind = rd.randint(3, len(color))
#     x,y = np.array(arr)[:,0], np.array(arr)[:,1]
#     ax.scatter(x,y, color=color[color_ind], marker=',', label='Mảnh '+ str(arr[0][3]), lw=1)

# # Đặt giới hạn cho trục 
# ax.set_xlim(0, 800)
# ax.set_ylim(0, 800)

# # Hiện các ghi chú trục
# ax.axis('on')
# ax.legend()
# plt.grid(True)
# plt.show()


# while True:
#     cv2.imshow('DatGach', image)
#     if cv2.waitKey(20) & 0xFF == 27:
#         break

# cv2.destroyAllWindows()
